In [1]:
from glob import glob
from pathlib import Path

from multiview_prediction_toolkit.cameras.derived_cameras import MetashapeCameraSet
from multiview_prediction_toolkit.meshes import TexturedPhotogrammetryMesh
from multiview_prediction_toolkit.config import DATA_FOLDER

from multiview_prediction_toolkit.segmentation import (
    LookUpSegmentor,
    SegmentorPhotogrammetryCameraSet,
)

2023-11-08 21:16:08:rastervision.pipeline.rv_config: WARNING - Root temporary directory cannot be used: /opt/data/tmp. Using root: /tmp/tmp_tub9qv4


In [9]:
# Processing parameters
HEIGHT_ABOVE_GROUND_THRESH = 2
MESH_DOWNSAMPLE_TARGET = 0.25
AGGREGATE_IMAGE_SCALE = 0.25
BUFFER_RADIUS_METER = 50
NUM_CLASSES = 10

## Raw input data
# Path to labels from field data
LABELS_FILENAME = Path(DATA_FOLDER, "example_Emerald_Point_data", "inputs", "labels.geojson")
# The mesh exported from Metashape
MESH_FILENAME = Path(DATA_FOLDER,"example_Emerald_Point_data", "inputs", "mesh.ply")
# The camera file exported from Metashape
CAMERAS_FILENAME = Path(DATA_FOLDER,"example_Emerald_Point_data", "inputs", "cameras.xml")
# The digital elevation map exported by Metashape
DTM_FILE = Path(DATA_FOLDER,"example_Emerald_Point_data", "inputs", "dtm.tif")
# The image folder used to create the Metashape project
IMAGE_FOLDER = Path(DATA_FOLDER,"example_Emerald_Point_data", "inputs", "images")

## Intermediate data from previous steps
PREDICTED_IMAGE_LABELS_FOLDER = Path(DATA_FOLDER, "example_Emerald_Point_data", "intermediate_results", "predicted_segmentations")

In [7]:
mesh = TexturedPhotogrammetryMesh(MESH_FILENAME)

In [3]:
# Create camera set
camera_set = MetashapeCameraSet(CAMERAS_FILENAME, IMAGE_FOLDER)
# Extract cameras near the training data
# TODO consider how to do the same thing with a mesh
training_camera_set = camera_set.get_subset_near_geofile(
    LABELS_FILENAME, buffer_radius_meters=BUFFER_RADIUS_METER
)

Loading GPS data for camera set: 100%|████████| 915/915 [00:10<00:00, 88.78it/s]


In [11]:
segmentor = LookUpSegmentor(
    base_folder=IMAGE_FOLDER,
    lookup_folder=PREDICTED_IMAGE_LABELS_FOLDER,
    num_classes=NUM_CLASSES,
)

segmentor_camera_set = SegmentorPhotogrammetryCameraSet(
    training_camera_set, segmentor=segmentor
)

In [ ]:
averaged_label_IDs, _, _ = mesh.aggregate_viewpoints_pytorch3d(
    segmentor_camera_set,
    image_scale=AGGREGATE_IMAGE_SCALE,
)

 15%|█████▉                                   | 110/754 [03:24<22:04,  2.06s/it]